### Imports and images

In [9]:
import os
from jita.tools.OllamaOCR import OllamaOCR
from jita.tools.zone_detector import HierarchicalFlyerDetector
from jita.settings.config import CASA_LEY_DATA, WINDOWED_CASA_LEY

folleto = list(CASA_LEY_DATA.glob("*.jpg"))[0]

In [10]:

print("=" * 70)
print("DETECTOR JERÁRQUICO OPTIMIZADO CON PADDING")
print("=" * 70)

# Configuración recomendada
detector = HierarchicalFlyerDetector(
    primary_divisions='auto',   # Auto-detecta columnas
    subdivisions=6,             # zonas por columna
    padding_percent=10,          # 10% de padding arriba/abajo
    preprocessing='balanced'    # Preprocesamiento equilibrado
)

zones = detector.detect_zones(
    folleto,
    f"{WINDOWED_CASA_LEY}/output_final",
    debug=True
)

# Mostrar resultados
print("\n" + "=" * 70)
print("ZONAS DETECTADAS:")
print("=" * 70)

for zone in zones:
    print(f"Zona {zone['id']:2d} | "
            f"Columna {zone['column']} | "
            f"Tamaño: {zone['width']:4d}x{zone['height']:4d}px | "
            f"Padding: +{zone['padding_applied']:3d}px | "
            f"→ {os.path.basename(zone['path'])}")

print("\n💡 Tips:")
print("  - Si las zonas se cortan texto, aumenta 'padding_percent' (ej: 10-15)")
print("  - Si detecta muy pocas columnas, usa primary_divisions=2 o 3")
print("  - Si necesitas más detalle, aumenta 'subdivisions' a 4 o 5")
print("  - Revisa las gráficas de proyección para ajustar detección")

DETECTOR JERÁRQUICO OPTIMIZADO CON PADDING
📸 Imagen preprocesada: 3247x1774px

🔍 Detectando divisiones principales...
   ⚠️  No se detectaron separadores, dividiendo en 2 columnas
   ✓ 2 columnas detectadas

🔍 Subdividiendo en 6 zonas c/u (padding: 10%)...
   → Columna 1: procesando...
      ✓ 6 zonas creadas
   → Columna 2: procesando...
      ✓ 6 zonas creadas

✅ Total: 12 zonas listas

📊 Visualización guardada: C:\Users\angel.merino\Documents\GitHub\jita\datos\casa_ley\octubre_2025\windowed/output_final\2_final_zones.jpg

ZONAS DETECTADAS:
Zona  1 | Columna 1 | Tamaño: 1623x 353px | Padding: + 58px | → zone_01.jpg
Zona  2 | Columna 1 | Tamaño: 1623x 354px | Padding: + 58px | → zone_02.jpg
Zona  3 | Columna 1 | Tamaño: 1623x 354px | Padding: + 58px | → zone_03.jpg
Zona  4 | Columna 1 | Tamaño: 1623x 353px | Padding: + 58px | → zone_04.jpg
Zona  5 | Columna 1 | Tamaño: 1623x 354px | Padding: + 58px | → zone_05.jpg
Zona  6 | Columna 1 | Tamaño: 1623x 325px | Padding: + 58px | → zone_06

### Qwen 2.5

In [2]:
qwen = OllamaOCR("qwen2.5vl:32b")

In [3]:
tiles = qwen.crop_by_zones(folleto, WINDOWED_CASA_LEY)

Successfully cropped 1 zones.


In [6]:
tiles[0]

'C:\\Users\\angel.merino\\Documents\\GitHub\\jita\\datos\\casa_ley\\octubre_2025\\windowed\\pagina_01_01102025_r0_c0.png'

In [7]:
print(qwen.extract_text(tiles[0]))

El folleto que se muestra es una promoción de ofertas válidas solo para el martes 30 de septiembre de 2025. Se trata de una lista de productos de frutas, verduras y otros alimentos, con precios especiales. A continuación, se analiza el contenido del folleto:

### **Información General**
- **Fecha de la Promoción**: Martes 30 de septiembre de 2025.
- **Tipo de Productos**: Frutas, verduras, hortalizas y otros alimentos.
- **Restricciones**: Se menciona que hay un límite de compra por cliente para algunos productos (por ejemplo, "máximo 5 kilos" o "máximo 5 piezas").

### **Productos y Precios**
#### **Primera Fila**
1. **PAPA**
   - **Variedad**: Russet
   - **Presentación**: A granel
   - **Precio**: $19.90 por kilo
   - **Restricción**: Máximo 5 kilos por cliente

2. **CHILE**
   - **Variedad**: Anaheim
   - **Precio**: $49.90 por kilo
   - **Restricción**: Máximo 5 kilos por cliente

3. **CIRUELA**
   - **Variedad**: Roja España
   - **Precio**: $79.90 por kilo
   - **Restricción**: 

### Mistral small

In [14]:
mistral = OllamaOCR("mistral-small3.2:24b")

In [ ]:
print(mistral.extract_text(folleto))

'El folleto que has compartido es una promoción de un supermercado llamado "Ley", celebrando su aniversario. La promoción es válida solo el martes 30 de septiembre de 2014. Aquí hay un resumen detallado de las secciones y ofertas destacadas:\n\n### Sección de Frutas y Verduras\n- **Papa Russet**: $19.90 por 10 kg\n- **Cebolla**: $14.90 por 5 kg\n- **Pepino**: $24.90 por 6 piezas\n- **Jícama**: $19.90 por 3 kg\n- **Zanahoria**: $15.90 por 5 kg\n- **Naranja**: $19.90 por 5 kg\n- **Limón**: $29.90 por 5 kg\n- **Tomate**: $14.90 por 3 kg\n- **Cebolla Blanca**: $19.90 por 5 kg\n\n### Sección de Carnes\n- **Carne de Res para Asar**: $208.90 por kg\n- **Carne de Res para Asar (Oferta)**: $123.90 por kg\n- **Pollo Fresco**: $67.90 por 8 piezas\n- **Pulpa de Res**: $187.90 por kg\n- **Menudo**: $98.90 por kg\n- **Pierna y Muslo de Pollo**: $109.90 por kg\n- **Bistec de Res**: $99.90 por kg\n- **Camarón**: $196.90 por kg\n\n### Sección de Productos Lácteos y Yogur\n- **Yogur y Postres**: 3 x 2 a